# MaxQuant (MQ) Output-Files

Compare a single experiment

Files compared:
1. `Summary.txt`
2. `mqpar.xml`
3. `peptides.txt`
4. `proteins.txt`

There is are many files more, where several files seem to be available in several times in different formats.

In [ ]:
folders = [folder for folder in  Path(FOLDER_MQ_TXT_DATA).iterdir()]

In [ ]:
folders_dict = {folder.name: folder for folder in sorted(folders)}
assert len(folders_dict) == len(folders), "Non unique file names"

In [ ]:
# w_file = widgets.Dropdown(options=[folder for folder in folders], description='View files')
w_file = widgets.Dropdown(options=folders_dict, description='View files')
w_file

In [ ]:
mq_output = MaxQuantOutputDynamic(w_file.value)
mq_output

Results will be saved in a subfolder under `vaep/project/data` using the name of the specified input-folder per default. Change to your liking:

> Go to the block you are interested in!

## MQ Summary files

In [ ]:
mq_output.summary.iloc[0].to_dict()

### File Handler

- dictionary of run name to run output folder
- find class with expected output folders

In [ ]:
# load_summary??

### Summaries

- aggregated in `vaep/project/erda_01_mq_aggregate_summaries.ipynb` 
    - file selection based on summaries for further analysis thereafter

In [ ]:
# paths_summaries = [str(folder / 'summary.txt') for folder in folders_dict.values()]

In [ ]:
# # if paths_summaries.files:
# if folders_dict:
# #     df, names, failed = process_files(handler_fct=load_summary, filepaths=paths_summaries.files, key=FOLDER_KEY, relative_to='paths_summaries.folder')
#     df, names, failed = process_files(handler_fct=load_summary, filepaths=paths_summaries, key=FOLDER_KEY, relative_to=None)
#     df.columns = names
#     print(f"Number of failed reads: {len(failed)}")
#     display(df)

In [ ]:
# # if paths_summaries.files:
# if paths_summaries:
#     df.to_csv(os.path.join(FOLDER_PROCESSED, 'all_summary_txt.csv'))
#     df.to_pickle(os.path.join(FOLDER_PROCESSED, 'all_summary_txt.pkl'))

- SIL - MS2 based on precursor which was a set of peaks
- PEAK - MS2 scan based on a single peak on precursor spectrum
- ISO - isotopic pattern detection


In [ ]:
# # if paths_summaries.files:
# if paths_summaries:
#     MS_spectra = df.loc[['MS', 'MS/MS Identified']].T.astype('int64')
#     mask  = MS_spectra['MS/MS Identified'] > 0
#     display(MS_spectra.loc[mask].describe())
#     MS_spectra.to_csv(os.path.join(FOLDER_PROCESSED, 'overview_stats.csv'))

## MaxQuant Parameter File

- partly in a separate subfolder
- mainly in run folders
- rebase on folders_dictionary (check for `.xml` files in all folders)

In [ ]:
mqpar_files = (Path(FOLDER_MQ_TXT_DATA) / 'mqpar_files')

mqpar_files  = list(mqpar_files.iterdir())
len(mqpar_files) # nested search needed

In [ ]:
# paths_parameters = PathsList(files=[file for file in all_files.files if '.xml' in file], folder=all_files.folder)
w_file = widgets.Dropdown(options=mqpar_files, description='Select a file')
w_file

### Parameter Files

In [ ]:
load_mqpar_xml??

In [ ]:
fname_mqpar_xml = os.path.join(FOLDER_PROCESSED, 'peptide_intensities.{}')

if mqpar_files:
    mqpar_files  = [str(file) for file in mqpar_files]
    df_mqpar, col_names, failed = process_files(handler_fct=load_mqpar_xml, filepaths=mqpar_files, key=FOLDER_KEY, relative_to=None) 
    df_mqpar.columns = col_names
    print(f"Number of failed reads: {len(failed)}")
    pd.set_option('max_rows', 160)
    display(df_mqpar)
    df_mqpar.to_pickle(fname_mqpar_xml.format("pkl"))

In [ ]:
# df_mqpar.T.convert_dtypes().dtypes


## Peptides

In [ ]:
paths_peptides = PathsList(files = [file for file in all_files.files if 'peptides.txt' in file], folder=all_files.folder)
# paths_peptides

In [ ]:
pd.set_option('max_columns', 60)

mq_output = MaxQuantOutputDynamic(
    folder=folders[random.randint(0, len(paths_peptides.files)-1)])
mq_output.peptides

In [ ]:
mq_output.peptides.Intensity # as is in peptides.txt, comma seperated thousands

### Create peptide intensity dumps for each MQ outputfolder

All folders are stored in a list

In [ ]:
# folders[:10]

Check if the output folder contains already parsed files

In [ ]:
import json

import config

with open(config.FN_FASTA_DB) as f:
    data_fasta = json.load(f)
print(f'Number of proteins in fasta file DB: {len(data_fasta)}')

In [ ]:
# 20170509_QE4_LC12_IAH_QC_MNT_HeLa_01 # trigger re-execution
# 20130408_QE6_LC5_KBS_MNT_QC_HeLa_02  # reversed protein leading razor protein

In [ ]:
%%time
FOLDER_PROCESSED = Path(FOLDER_PROCESSED)
set_previously_loaded =  {folder.name for folder in FOLDER_PROCESSED.iterdir()}

FORCE = True

for folder in folders:
    if folder.name in set_previously_loaded and not FORCE and (folder / '0_completness_all_genes.json').exists():
        pass
    else:
        logger.info('\n\nProcess: {folder.name}')
        print(f"Process: {folder.name}")
        mq_output = MaxQuantOutputDynamic(folder)
        peptide_extractor = ExtractFromPeptidesTxt(
            out_folder=FOLDER_PROCESSED, mq_output_object=mq_output, fasta_db=data_fasta)
        completeness_per_gene = peptide_extractor()

In [ ]:
# %debug

## Theoretial Peptides from used fasta-file

> `01_explore_FASTA.ipynb` (formely `01_FASTA_tryptic_digest.ipynb`)

- check if peptides are part of theoretical peptides